In [46]:
# Import necessary libraries
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
#%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image
from time import sleep
import os
import numpy as np
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate
from sklearn import tree
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE

In [47]:
df = pd.read_csv('Desktop/Speed_Dating_Clean.csv')

In [48]:
match = pd.DataFrame(df['match'])
data_X = df.drop(['match','decision','decision_o','Unnamed: 0'],axis =1)

In [49]:
# change gender into binary
def gender(value): 
    if value == 'female': 
        return 0 
    else: 
        return 1
male = data_X['gender'].map(gender)
data_X['gender'] = male

In [50]:
def d_d_age(value): 
    if value == '[2-3]': 
        return 2
    if value == '[4-6]': 
        return 3
    if value == '[0-1]': 
        return 1
    if value == '[7-37]': 
        return 4
dage = data_X['d_d_age'].map(d_d_age)
data_X['d_d_age'] = dage

In [51]:
def race(value): 
    if value == 'asian/pacific islander/asian-american': 
        return 1
    if value == 'black/african american': 
        return 2
    if value == 'european/caucasian-american': 
        return 3
    if value =='latino/hispanic american': 
        return 4
    else: 
        return 5
racee = data_X['race'].map(race)
data_X['race'] = racee

In [52]:
race_o =data_X['race_o'].map(race)
data_X['race_o'] = race_o

In [53]:
study1 = list(data_X['field'].value_counts().index)
def study(value): 
    if value in study1: 
        return study1.index(value)
study_idx = data_X['field'].map(study)
data_X['field'] = study_idx

In [54]:
data_X[:5]

gender   age  age_o  d_age  d_d_age  race  race_o  samerace  \
0       0  21.0   27.0      6        3     1       3         0   
1       0  21.0   22.0      1        1     1       3         0   
2       0  21.0   22.0      1        1     1       1         1   
3       0  21.0   23.0      2        2     1       3         0   
4       0  21.0   24.0      3        2     1       4         0   

   importance_same_race  importance_same_religion ...   concerts  music  \
0                   2.0                       4.0 ...       10.0    9.0   
1                   2.0                       4.0 ...       10.0    9.0   
2                   2.0                       4.0 ...       10.0    9.0   
3                   2.0                       4.0 ...       10.0    9.0   
4                   2.0                       4.0 ...       10.0    9.0   

   shopping  yoga  interests_correlate  expected_happy_with_sd_people  \
0       8.0   1.0                 0.14                            3.0   
1       8.0   1.0                 0.54                            3.0   
2       8.0   1.0                 0.16                            3.0   
3       8.0   1.0                 0.61                            3.0   
4       8.0   1.0                 0.21                            3.0   

   expected_num_matches  like  guess_prob_liked  met  
0                   4.0   7.0               6.0  0.0  
1                   4.0   7.0               5.0  1.0  
2                   4.0   7.0               0.0  1.0  
3                   4.0   7.0               6.0  0.0  
4                   4.0   6.0               6.0  0.0  

[5 rows x 63 columns]

# Under - Sampling to deal with class imbalance

In [55]:
data_X2 = data_X.copy()
data_X2['match'] = match 

In [56]:
# first you need to select the ones from majority class and pick it at random
no_match = len(data_X2[data_X2['match'] == 0])
no_match_indices = data_X2[data_X2.match == 0].index
random_indices = np.random.choice(no_match_indices,no_match, replace=False)

In [57]:
# get the matched indices and then concatenate to get final
match_indices = data_X2[data_X2.match == 1].index
under_sample_indices = np.concatenate([match_indices,random_indices])
under_sample = data_X2.loc[under_sample_indices]

In [58]:
X_under = under_sample.loc[:,under_sample.columns != 'match']
y_under = under_sample.loc[:,under_sample.columns == 'match']
X_under_train, X_under_test, y_under_train, y_under_test = train_test_split(X_under,y_under,test_size = 0.3, random_state = 0)

In [59]:
clf = tree.DecisionTreeClassifier(criterion = 'entropy')
clf = clf.fit(X_under_train, y_under_train)

In [60]:
preds = clf.predict(X_under_test)
print('Accuracy on test data is:',(accuracy_score(y_under_test,preds)))

Accuracy on test data is: 0.7944038929440389


In [61]:
print('The confusion matrix is:\n',confusion_matrix(y_under_test,preds))
print(classification_report(y_under_test,preds))

The confusion matrix is:
 [[1799  268]
 [ 239  160]]
              precision    recall  f1-score   support

           0       0.88      0.87      0.88      2067
           1       0.37      0.40      0.39       399

   micro avg       0.79      0.79      0.79      2466
   macro avg       0.63      0.64      0.63      2466
weighted avg       0.80      0.79      0.80      2466



# SMOTE Analysis

In [62]:
x_train, x_val, y_train, y_val = train_test_split(data_X, match,
                                                  test_size = .1,
                                                  random_state=12)

In [63]:
sm = SMOTE(random_state=12, ratio = 1.0)
x_train_res, y_train_res = sm.fit_sample(x_train, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [64]:
from sklearn.ensemble import RandomForestClassifier
clf_rf = RandomForestClassifier(n_estimators=25, random_state=12)
clf_rf.fit(x_train_res, y_train_res)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=None,
            oob_score=False, random_state=12, verbose=0, warm_start=False)

In [68]:
# this is the result that we would get on the validation set
print('The confusion matrix is:\n',confusion_matrix(y_val,clf_rf.predict(x_val)))
print(classification_report(y_val, clf_rf.predict(x_val)))

The confusion matrix is:
 [[671  34]
 [ 63  54]]
              precision    recall  f1-score   support

           0       0.91      0.95      0.93       705
           1       0.61      0.46      0.53       117

   micro avg       0.88      0.88      0.88       822
   macro avg       0.76      0.71      0.73       822
weighted avg       0.87      0.88      0.87       822



In [69]:
# this is the result that we would get on the test set which is the entire dataset 
print('The confusion matrix is:\n',confusion_matrix(match,clf_rf.predict(data_X)))
print(classification_report(match, clf_rf.predict(data_X)))

The confusion matrix is:
 [[6834   36]
 [  64 1286]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      6870
           1       0.97      0.95      0.96      1350

   micro avg       0.99      0.99      0.99      8220
   macro avg       0.98      0.97      0.98      8220
weighted avg       0.99      0.99      0.99      8220

